In [1]:
import numpy as np
import pandas as pd

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ods-churn-24/currency_rk.csv
/kaggle/input/ods-churn-24/report_dates.csv
/kaggle/input/ods-churn-24/mcc_codes.csv
/kaggle/input/ods-churn-24/clients.csv
/kaggle/input/ods-churn-24/train.csv
/kaggle/input/ods-churn-24/sample_submit_naive.csv
/kaggle/input/ods-churn-24/transactions.csv


In [2]:
import pandas as pd
import numpy as np
# from sksurv.ensemble import RandomSurvivalForest
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import random
import warnings
from IPython.display import display, HTML

# pd.set_option('display.max_columns', None)
warnings.simplefilter('ignore')
# pd.options.display.max_columns = 100
# pd.options.display.max_rows = 100
random.seed(42)
np.random.seed(42)

In [3]:
clients = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
report_dates = pd.read_csv('/kaggle/input/ods-churn-24/report_dates.csv', parse_dates=['report_dt'])
transactions = pd.read_csv('/kaggle/input/ods-churn-24/transactions.csv', parse_dates=['transaction_dttm'])
currency_rk = pd.read_csv('/kaggle/input/ods-churn-24/currency_rk.csv')
currency_mult = pd.DataFrame({'currency_rk': [0,1,2,3], 'mult':[24,1,100,90]})
mcc_codes = pd.read_csv('/kaggle/input/ods-churn-24/mcc_codes.csv')
sample_submit_naive = pd.read_csv('/kaggle/input/ods-churn-24/sample_submit_naive.csv')
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')

transactions = transactions.merge(currency_mult, how='left')
transactions.transaction_amt = transactions.transaction_amt * transactions.mult
transactions.drop(columns=['mult'], inplace=True)

In [4]:
transactions['time_delta'] = transactions.groupby('user_id')['transaction_dttm'].diff().dt.total_seconds().fillna(0) / (3600*24)
transactions['transaction_amt'] = transactions['time_delta']
transactions = transactions.drop(['time_delta'], axis=1)

#### FeatureTools

In [5]:
import featuretools as ft

# Load the datasets
train_df = clients[['user_id']].copy()
transactions_df = transactions.drop(['currency_rk'], axis=1).copy()

# train_df = clients[['user_id']][0:500].copy()
# transactions_df = transactions[0:5000].drop(['currency_rk'], axis=1).copy()

# Convert transaction_dttm to datetime
transactions_df['transaction_dttm'] = pd.to_datetime(transactions_df['transaction_dttm'])

# Create an EntitySet
es = ft.EntitySet(id='user_transactions')

# Add the transactions dataframe to the EntitySet
es = es.add_dataframe(
    dataframe_name='transactions',
    dataframe=transactions_df,
    index='transaction_id',
    time_index='transaction_dttm',
    logical_types={
        'mcc_code': 'Categorical',
        'transaction_amt': 'Double',  # Corrected logical type
        'transaction_dttm': 'Datetime'
    }
)

# Add the train dataframe to the EntitySet
es = es.add_dataframe(
    dataframe_name='train',
    dataframe=train_df,
    index='user_id'
)

# Correctly defining the relationship
es = es.add_relationships([("train", "user_id", "transactions", "user_id")])

# Set interesting values (example)
# transactions_df['mcc_code'].value_counts().nlargest(10).index.tolist() could be used to find interesting mcc_codes
# Here, you manually define them or use a method to find them dynamically
interesting_mcc_codes = transactions['mcc_code'].value_counts(normalize=True).index.tolist()  # Example MCC codes, replace with relevant codes for your case
es['transactions']['mcc_code'].interesting_values = interesting_mcc_codes

# Automatically generate features with expanded primitives and max_depth
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name='train',
    agg_primitives=[
        'sum', 'mean', 'max', 'min', 'count', 'percent_true', 'num_unique', 'mode', 'skew',
        'trend', 'std'
    ],  # Expanded list of aggregation primitives
    trans_primitives=[
        'day', 'month', 'year', 'weekday', 'is_weekend',
        'time_since_previous', 'cum_sum', 'percentile', 'diff'
    ],  # Expanded list of transformation primitives
    where_primitives=['sum', 'mean', 'max'],  # Conditional aggregations
    max_depth=3,  # Increased max_depth for more complex features
)

In [6]:
main_ft = clients[['user_id']].copy()
main_ft = main_ft.merge(feature_matrix, how='left', on='user_id')
main_ft

,user_id,COUNT(transactions),MAX(transactions.transaction_amt),MEAN(transactions.transaction_amt),MIN(transactions.transaction_amt),MODE(transactions.mcc_code),NUM_UNIQUE(transactions.mcc_code),SKEW(transactions.transaction_amt),STD(transactions.transaction_amt),SUM(transactions.transaction_amt),...,PERCENTILE(STD(transactions.PERCENTILE(transaction_amt))),PERCENTILE(STD(transactions.TIME_SINCE_PREVIOUS(transaction_dttm))),PERCENTILE(SUM(transactions.CUM_SUM(transaction_amt))),PERCENTILE(SUM(transactions.DIFF(transaction_amt))),PERCENTILE(SUM(transactions.PERCENTILE(transaction_amt))),PERCENTILE(SUM(transactions.TIME_SINCE_PREVIOUS(transaction_dttm))),"PERCENTILE(TREND(transactions.CUM_SUM(transaction_amt), transaction_dttm))","PERCENTILE(TREND(transactions.DIFF(transaction_amt), transaction_dttm))","PERCENTILE(TREND(transactions.PERCENTILE(transaction_amt), transaction_dttm))","PERCENTILE(TREND(transactions.TIME_SINCE_PREVIOUS(transaction_dttm), transaction_dttm))"
0,3,11,30.108970,9.668826,0.0,0,4,0.755633,10.674091,106.357083,...,0.998927,0.075479,0.015927,0.658937,0.005667,0.033969,0.378906,0.750406,0.882500,0.351208
1,9,90,21.808750,2.014418,0.0,1,22,3.395471,3.747738,181.297581,...,0.611010,0.833292,0.124073,0.800292,0.544865,0.721422,0.019271,0.568333,0.085687,0.073656
2,13,22,21.620289,7.652659,0.0,3,4,0.547426,7.302019,168.358495,...,0.831521,0.857802,0.160229,0.963771,0.163208,0.281443,0.692865,0.788438,0.797365,0.143000
3,37,315,6.925602,0.568326,0.0,2,28,3.507543,0.885702,179.022535,...,0.151469,0.627594,0.828844,0.050375,0.903344,0.909474,0.617687,0.549146,0.115052,0.197500
4,41,16,31.168206,9.575446,0.0,3,5,0.920947,12.859846,153.207130,...,0.967229,0.814073,0.009604,0.838271,0.054667,0.202208,0.124698,0.796312,0.793396,0.038458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,37,53.427824,3.343917,0.0,1,12,4.690475,9.405368,123.724942,...,0.634302,0.989729,0.492281,0.552656,0.249000,0.427542,0.240563,0.961823,0.937344,0.980010
95996,562205,151,8.054167,1.181126,0.0,11,15,2.007381,1.576596,178.350035,...,0.386510,0.812937,0.859542,0.435823,0.718115,0.792807,0.122146,0.382083,0.521052,0.903667
95997,562312,56,37.073229,2.821125,0.0,1,7,4.430741,5.676918,157.983009,...,0.563990,0.862521,0.614688,0.453865,0.410573,0.541943,0.172104,0.248417,0.791240,0.879896
95998,562721,85,25.439109,2.057463,0.0,3,11,3.780682,3.943918,174.884363,...,0.326813,0.969094,0.723437,0.520417,0.539562,0.871708,0.176052,0.672708,0.770146,0.853625


#### TSFresh

In [7]:
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.utilities.dataframe_functions import impute

# Load the transaction data
transactions_df = transactions.drop(['currency_rk'], axis=1).copy()

# transactions_df = transactions[0:5000].drop(['currency_rk'], axis=1).copy()

# Ensure 'transaction_dttm' is a datetime type, assuming it's already in an appropriate datetime format
transactions_df['transaction_dttm'] = pd.to_datetime(transactions_df['transaction_dttm'])

# Setting up the data in a format suitable for TSFresh
# Assuming 'user_id' is in your transactions_df to identify different users
# We will use 'transaction_dttm' as the time index
transactions_df_sorted = transactions_df.sort_values(by=['user_id', 'transaction_dttm'])

# Extract features
extraction_settings = ComprehensiveFCParameters()

# The 'column_id' is assumed to be 'user_id' to treat each user's data as a separate time series
# The 'column_sort' is 'transaction_dttm' to sort transactions in time order
# 'column_value' could be 'transaction_amt' if you're interested in extracting features from the transaction amounts
extracted_features = extract_features(
    transactions_df_sorted,
    column_id='user_id',
    column_sort='transaction_dttm',
    column_value='transaction_amt',  # Or any other column you want to analyze
    default_fc_parameters=extraction_settings,
    impute_function=impute  # Impute missing values generated during feature extraction
)

Feature Extraction: 100%|██████████| 10/10 [2:49:09<00:00, 1014.99s/it]


In [8]:
extracted_features['user_id'] = extracted_features.index

main_tsf = clients[['user_id']].copy()
main_tsf = main_tsf.merge(extracted_features, how='left', on='user_id')
main_tsf

,user_id,transaction_amt__variance_larger_than_standard_deviation,transaction_amt__has_duplicate_max,transaction_amt__has_duplicate_min,transaction_amt__has_duplicate,transaction_amt__sum_values,transaction_amt__abs_energy,transaction_amt__mean_abs_change,transaction_amt__mean_change,transaction_amt__mean_second_derivative_central,...,transaction_amt__fourier_entropy__bins_5,transaction_amt__fourier_entropy__bins_10,transaction_amt__fourier_entropy__bins_100,transaction_amt__permutation_entropy__dimension_3__tau_1,transaction_amt__permutation_entropy__dimension_4__tau_1,transaction_amt__permutation_entropy__dimension_5__tau_1,transaction_amt__permutation_entropy__dimension_6__tau_1,transaction_amt__permutation_entropy__dimension_7__tau_1,transaction_amt__query_similarity_count__query_None__threshold_0.0,transaction_amt__mean_n_absolute_max__number_of_maxima_7
0,3,1.0,0.0,0.0,0.0,106.357083,2167.710212,13.841704,0.001752,-0.500139,...,0.867563,1.242453,1.791759,1.522955,2.079442,1.945910,1.791759,1.609438,0.0,15.186197
1,9,1.0,0.0,0.0,0.0,181.297581,1615.262120,2.766730,0.002549,-0.107195,...,1.439129,2.112010,3.523285,1.748389,2.985571,4.020933,4.410033,4.430817,0.0,12.624401
2,13,1.0,0.0,0.0,0.0,168.358495,2408.099311,8.346281,0.097319,-0.722573,...,0.983088,1.560710,2.253858,1.496416,2.187271,2.659323,2.833213,2.772589,0.0,16.759038
3,37,0.0,0.0,0.0,1.0,179.022535,348.066209,0.719598,0.001803,-0.010270,...,0.883934,1.514821,3.563978,1.788860,3.152388,4.526527,5.410035,5.669938,0.0,4.829053
4,41,1.0,0.0,0.0,0.0,153.207130,3947.661289,17.607971,0.153914,-2.094508,...,1.303092,1.581094,1.889159,1.772139,2.458311,2.484907,2.397895,2.302585,0.0,20.866047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,1.0,0.0,0.0,0.0,123.724942,3598.320207,4.014127,1.484106,0.629133,...,0.409459,0.409459,1.789748,1.725368,2.664724,3.160436,3.335771,3.389268,0.0,14.779874
95996,562205,1.0,0.0,0.0,0.0,178.350035,583.502001,1.417290,0.003294,-0.003962,...,1.475280,2.130510,3.712606,1.785925,3.129595,4.379572,4.812694,4.948052,0.0,6.251070
95997,562312,1.0,0.0,0.0,0.0,157.983009,2218.196900,4.003792,0.041153,0.005966,...,1.368750,1.926490,3.176083,1.781398,2.920909,3.711308,3.877461,3.912023,0.0,14.209306
95998,562721,1.0,0.0,0.0,0.0,174.884363,1666.395117,2.884005,0.018749,-0.173971,...,1.154604,1.795992,3.446708,1.776566,3.055601,3.981430,4.278055,4.351900,0.0,13.109600


#### Selection

In [9]:
data = main_ft.merge(train[['user_id', 'target']], how='left')
data = data[~data.target.isna()]
labels = data['target']
data = data.drop(columns=['target', 'user_id'])

print(main_ft.shape)

(96000, 174)


In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold

# Load the datasets
features_df = data.copy()
target_df = labels.copy()

# Assuming 'user_id' is the common key and both datasets are aligned
# If not, you might need to merge or align them based on your specific needs

# Step 1: Remove features with a big share of Missing Values
# Let's assume "a big share" means more than 60% missing
threshold = 0.6  # 60%
features_df = features_df.loc[:, features_df.isnull().mean() < threshold]

# Step 2: Remove features with Single Unique Value
features_df = features_df.loc[:, features_df.apply(pd.Series.nunique) != 1]

# Step 3: Remove duplicate features
# Transpose the dataframe, drop duplicate rows (now columns), and transpose back
features_df = features_df.T.drop_duplicates().T

# Step 4: Remove collinear features
# Calculate the correlation matrix and remove one of two features with correlation higher than a threshold
corr_matrix = features_df.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
features_df = features_df.drop(columns=to_drop)

# If necessary, align the target with the features based on 'user_id' or another identifier
# This step is skipped here but can be performed as needed

# Step 5: Optionally, remove features with low variance
# This requires fitting the data, so ensure to handle missing values if any remain
# Fill missing values with 0, mean, median, or any other method as appropriate for your dataset
features_df.fillna(0, inplace=True)  # Example: filling with 0
selector = VarianceThreshold(threshold=0.01)  # Adjust threshold as needed
features_reduced = selector.fit_transform(features_df)

# Convert back to DataFrame if necessary, for further use
features_final = pd.DataFrame(features_reduced, columns=features_df.columns[selector.get_support()])
good_cols = features_final.columns

main_ft = main_ft[['user_id']+list(good_cols)]
print(main_ft.shape)

(96000, 87)


In [11]:
data = main_tsf.merge(train[['user_id', 'target']], how='left')
data = data[~data.target.isna()]
labels = data['target']
data = data.drop(columns=['target', 'user_id'])

print(main_tsf.shape)

(96000, 784)


In [12]:
# Load the datasets
features_df = data.copy()
target_df = labels.copy()

# Assuming 'user_id' is the common key and both datasets are aligned
# If not, you might need to merge or align them based on your specific needs

# Step 1: Remove features with a big share of Missing Values
# Let's assume "a big share" means more than 60% missing
threshold = 0.6  # 60%
features_df = features_df.loc[:, features_df.isnull().mean() < threshold]

# Step 2: Remove features with Single Unique Value
features_df = features_df.loc[:, features_df.apply(pd.Series.nunique) != 1]

# Step 3: Remove duplicate features
# Transpose the dataframe, drop duplicate rows (now columns), and transpose back
features_df = features_df.T.drop_duplicates().T

# Step 4: Remove collinear features
# Calculate the correlation matrix and remove one of two features with correlation higher than a threshold
corr_matrix = features_df.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
features_df = features_df.drop(columns=to_drop)

# If necessary, align the target with the features based on 'user_id' or another identifier
# This step is skipped here but can be performed as needed

# Step 5: Optionally, remove features with low variance
# This requires fitting the data, so ensure to handle missing values if any remain
# Fill missing values with 0, mean, median, or any other method as appropriate for your dataset
features_df.fillna(0, inplace=True)  # Example: filling with 0
selector = VarianceThreshold(threshold=0.01)  # Adjust threshold as needed
features_reduced = selector.fit_transform(features_df)

# Convert back to DataFrame if necessary, for further use
features_final = pd.DataFrame(features_reduced, columns=features_df.columns[selector.get_support()])
good_cols = features_final.columns

main_tsf = main_tsf[['user_id']+list(good_cols)]
print(main_tsf.shape)

(96000, 653)


#### Importance

In [13]:
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
train = main_ft.merge(train[['user_id', 'target']], how='left')
train = train[~train.target.isna()]
train = train.sort_values('user_id').reset_index(drop=True)

cat_cols = main_ft.select_dtypes(exclude=["number","bool_","object_"]).columns.to_list()

# Обучение модельки для того чтобы получить важные фичи

model = CatBoostClassifier(
    iterations = 1400,
    depth=5,
    learning_rate=0.03,
    
    eval_metric='AUC',
    cat_features = cat_cols,
    thread_count=6,
    early_stopping_rounds=200,
)
model.fit(train.drop(['user_id', 'target'], axis=1), train['target'], verbose=100)


df_imp = pd.DataFrame({
    'name': train.drop(['user_id', 'target'], axis=1).columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

df_imp = df_imp[df_imp['imp'] > 0.15] # Берем все фичи, у которых важность больше 0.3
df_imp_050 = df_imp[df_imp['imp'] > 0.50]
df_imp_055 = df_imp[df_imp['imp'] > 0.55]
df_imp_060 = df_imp[df_imp['imp'] > 0.60]
df_imp_065 = df_imp[df_imp['imp'] > 0.65]

good_cols = df_imp['name'].tolist()
good_cols_050 = df_imp_050['name'].tolist()
good_cols_055 = df_imp_055['name'].tolist()
good_cols_060 = df_imp_060['name'].tolist()
good_cols_065 = df_imp_065['name'].tolist()

pred = model.predict_proba(train.drop(['user_id', 'target'], axis=1))[:, 1]
print(metrics.roc_auc_score(train['target'], pred))

TBB Warning: The number of workers is currently limited to 3. The request for 5 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



0:	total: 151ms	remaining: 3m 30s
100:	total: 8.03s	remaining: 1m 43s
200:	total: 15.8s	remaining: 1m 34s
300:	total: 24s	remaining: 1m 27s
400:	total: 31.3s	remaining: 1m 18s
500:	total: 38.7s	remaining: 1m 9s
600:	total: 46.3s	remaining: 1m 1s
700:	total: 54.1s	remaining: 54s
800:	total: 1m 1s	remaining: 46s
900:	total: 1m 9s	remaining: 38.3s
1000:	total: 1m 16s	remaining: 30.7s
1100:	total: 1m 25s	remaining: 23.2s
1200:	total: 1m 32s	remaining: 15.4s
1300:	total: 1m 40s	remaining: 7.65s
1399:	total: 1m 48s	remaining: 0us
0.8192484750410257


In [14]:
df_imp[0:50]

,name,imp
33,CUM_SUM(COUNT(transactions)),12.025009
48,CUM_SUM(MIN(transactions.TIME_SINCE_PREVIOUS(t...,5.706677
8,MAX(transactions.CUM_SUM(transaction_amt)),5.577979
4,NUM_UNIQUE(transactions.mcc_code),4.629560
43,PERCENTILE(NUM_UNIQUE(transactions.mcc_code)),3.866540
46,PERCENTILE(SUM(transactions.transaction_amt)),2.693195
7,SUM(transactions.transaction_amt),2.542282
5,SKEW(transactions.transaction_amt),2.179336
44,PERCENTILE(SKEW(transactions.transaction_amt)),2.061938
25,SKEW(transactions.PERCENTILE(transaction_amt)),1.754413


In [15]:
main_ft = main_ft[['user_id']+good_cols]
main_ft_050 = main_ft[['user_id']+good_cols_050]
main_ft_055 = main_ft[['user_id']+good_cols_055]
main_ft_060 = main_ft[['user_id']+good_cols_060]
main_ft_065 = main_ft[['user_id']+good_cols_065]

In [16]:
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
train = main_tsf.merge(train[['user_id', 'target']], how='left')
train = train[~train.target.isna()]
train = train.sort_values('user_id').reset_index(drop=True)

cat_cols = main_tsf.select_dtypes(exclude=["number","bool_","object_"]).columns.to_list()

# Обучение модельки для того чтобы получить важные фичи

model = CatBoostClassifier(
    iterations = 1400,
    depth=5,
    learning_rate=0.03,
    
    eval_metric='AUC',
#     cat_features = cat_cols,
    thread_count=6,
    early_stopping_rounds=200,
)
model.fit(train.drop(['user_id', 'target'], axis=1), train['target'], verbose=100)


df_imp = pd.DataFrame({
    'name': train.drop(['user_id', 'target'], axis=1).columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

df_imp = df_imp[df_imp['imp'] > 0.15] # Берем все фичи, у которых важность больше 0.3
df_imp_050 = df_imp[df_imp['imp'] > 0.50]
df_imp_055 = df_imp[df_imp['imp'] > 0.55]
df_imp_060 = df_imp[df_imp['imp'] > 0.60]
df_imp_065 = df_imp[df_imp['imp'] > 0.65]

good_cols = df_imp['name'].tolist()
good_cols_050 = df_imp_050['name'].tolist()
good_cols_055 = df_imp_055['name'].tolist()
good_cols_060 = df_imp_060['name'].tolist()
good_cols_065 = df_imp_065['name'].tolist()

pred = model.predict_proba(train.drop(['user_id', 'target'], axis=1))[:, 1]
print(metrics.roc_auc_score(train['target'], pred))

0:	total: 183ms	remaining: 4m 16s
100:	total: 17.6s	remaining: 3m 45s
200:	total: 32.6s	remaining: 3m 14s
300:	total: 47.6s	remaining: 2m 53s
400:	total: 1m 1s	remaining: 2m 32s
500:	total: 1m 15s	remaining: 2m 14s
600:	total: 1m 28s	remaining: 1m 57s
700:	total: 1m 42s	remaining: 1m 42s
800:	total: 1m 55s	remaining: 1m 26s
900:	total: 2m 8s	remaining: 1m 11s
1000:	total: 2m 31s	remaining: 1m
1100:	total: 2m 53s	remaining: 47s
1200:	total: 3m 6s	remaining: 30.9s
1300:	total: 3m 20s	remaining: 15.3s
1399:	total: 3m 34s	remaining: 0us
0.832650047191202


In [17]:
df_imp[0:50]

,name,imp
3,transaction_amt__sum_values,7.471908
642,transaction_amt__energy_ratio_by_chunks__num_s...,1.361381
643,transaction_amt__ratio_beyond_r_sigma__r_0.5,1.126910
154,"transaction_amt__change_quantiles__f_agg_""mean...",0.862400
12,transaction_amt__variation_coefficient,0.853537
50,transaction_amt__autocorrelation__lag_1,0.816339
177,"transaction_amt__fft_coefficient__attr_""real""_...",0.685759
43,transaction_amt__quantile__q_0.2,0.680621
375,"transaction_amt__fft_coefficient__attr_""abs""__...",0.627439
69,transaction_amt__binned_entropy__max_bins_10,0.548280


In [18]:
main_tsf = main_tsf[['user_id']+good_cols]
main_tsf_050 = main_tsf[['user_id']+good_cols_050]
main_tsf_055 = main_tsf[['user_id']+good_cols_055]
main_tsf_060 = main_tsf[['user_id']+good_cols_060]
main_tsf_065 = main_tsf[['user_id']+good_cols_065]

In [19]:
main_ft.to_csv('main_ft.csv', index=False)
main_tsf.to_csv('main_tsf.csv', index=False)

In [20]:
main_ft_050.to_csv('main_ft_050.csv', index=False)
main_ft_055.to_csv('main_ft_055.csv', index=False)
main_ft_060.to_csv('main_ft_060.csv', index=False)
main_ft_065.to_csv('main_ft_065.csv', index=False)
main_tsf_050.to_csv('main_tsf_050.csv', index=False)
main_tsf_055.to_csv('main_tsf_055.csv', index=False)
main_tsf_060.to_csv('main_tsf_060.csv', index=False)
main_tsf_065.to_csv('main_tsf_065.csv', index=False)